## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Oct. 20th Oct. 27th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,October-25-2024,Condo/Co-op,700 NE 26th Ter #2302,Miami,FL,33137.0,2300000.0,2.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11672925,N,Y,25.802013,-80.185776


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,2024-10-25,Condo/Co-op,700 NE 26th Ter #2302,Miami,FL,33137.0,2300000.0,2.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11672925,N,Y,25.802013,-80.185776
17,PAST SALE,2024-10-25,Condo/Co-op,55 SW 9th St #2804,Miami,FL,33130.0,639000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/55-SW-9th-St-3...,MARMLS,A11664845,N,Y,25.765661,-80.194394
110,PAST SALE,2024-10-25,Condo/Co-op,230 NE 174th St #315,Sunny Isles Beach,FL,33160.0,410000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/23...,Beaches MLS,RX-10942796,N,Y,25.936714,-80.124549
102,PAST SALE,2024-10-25,Condo/Co-op,1035 West Ave #404,Miami Beach,FL,33139.0,350000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1035-Wes...,MARMLS,A11547639,N,Y,25.781266,-80.142037
101,PAST SALE,2024-10-25,Condo/Co-op,1750 N Bayshore Dr #5103,Miami,FL,33132.0,360000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1750-N-Bayshor...,MARMLS,A11547948,N,Y,25.792509,-80.187129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,PAST SALE,2024-10-21,Condo/Co-op,1470 NE 125th Ter #904,North Miami,FL,33161.0,145000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/1470-NE-...,MARMLS,A11627948,N,Y,25.891300,-80.169161
81,PAST SALE,2024-10-21,Condo/Co-op,900 Brickell Key Blvd #1805,Miami,FL,33131.0,1040000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11595145,N,Y,25.768844,-80.185937
83,PAST SALE,2024-10-21,Condo/Co-op,460 E 23rd St #403,Hialeah,FL,33013.0,235000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/460-E-23rd-S...,MARMLS,A11616240,N,Y,25.842437,-80.272650
85,PAST SALE,2024-10-21,Condo/Co-op,5249 NW 7th St #401,Miami,FL,33126.0,290000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/5249-NW-7th-St...,MARMLS,A11607662,N,Y,25.778950,-80.281257


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

112


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$782,730


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$478


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$87,665,799


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami Dade
Input Previous Week Condo Sales Total (number units sold):
111
Input Previous Week Condo Average Sales Price:
706659
Input Previous Week Condo Average PSF:
479
Input Previous Week Condo Sales total (ex: 198_000_000)
78_400_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-21,Condo/Co-op,1451 Brickell Ave Ph 56,Miami,FL,33131.0,33000000.0,5.0,7.5,ECHO BRICKELL CONDO,11068.0,NaN,2017.0,NaN,2982.0,17812.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11515752,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Chad Carroll,"Compass Florida, LLC",Matthew Dugow,"Compass Florida, LLC.",Stephan Hermida,Engel & Völkers Miami,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-21,Condo/Co-op,1451 Brickell Ave Ph 56,Miami,FL,33131.0,33000000.0,5.0,7.5,ECHO BRICKELL CONDO,11068.0,NaN,2017.0,NaN,2982.0,17812.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11515752,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Chad Carroll,"Compass Florida, LLC",Matthew Dugow,"Compass Florida, LLC.",Stephan Hermida,Engel & Völkers Miami,None,None,1,orange
1,PAST SALE,2024-10-25,Condo/Co-op,700 NE 26th Ter #2302,Miami,FL,33137.0,2300000.0,2.0,3.5,EDGEWATER CONDO,2347.0,NaN,2021.0,NaN,980.0,3696.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11672925,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Chad Carroll,"Compass Florida, LLC",None,None,Rana Tilllawi,"Compass Florida, LLC.",None,None,2,blue
2,PAST SALE,2024-10-22,Condo/Co-op,1451 Brickell Ave #3001,Miami,FL,33131.0,2300000.0,2.0,3.0,ECHO BRICKELL CONDO,1637.0,NaN,2017.0,NaN,1405.0,3785.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11537487,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Allan Filgueiras,AF Realty,None,None,None,None,None,None,3,blue
3,PAST SALE,2024-10-21,Condo/Co-op,900 Brickell Key Blvd #401,Miami,FL,33131.0,2000000.0,3.0,3.5,ASIA CONDO,2065.0,NaN,2008.0,NaN,969.0,3412.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11613137,N,Y,25.768844,-80.185937,https://www.redfin.com/FL/Miami/900-Brickell-K...,Rita Herrera,London Foster Realty,None,None,Rita Herrera,London Foster Realty,None,None,4,blue
4,PAST SALE,2024-10-25,Condo/Co-op,480 NE 31st St #4801,Miami,FL,33137.0,1950000.0,4.0,4.5,GRAN PARAISO CONDO,1871.0,NaN,2019.0,NaN,1042.0,1920.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/480-NE-31st-St...,MARMLS,A11448155,N,Y,25.806186,-80.187176,https://www.redfin.com/FL/Miami/480-NE-31st-St...,Fabio Visconti,"Rising Global Realty, LLC.",None,None,Adam Redolfi,Alure Capital,None,None,5,blue
5,PAST SALE,2024-10-21,Condo/Co-op,2575 S Bayshore Dr Unit 9B,Miami,FL,33133.0,1500000.0,3.0,2.0,THE COCONUT GROVE BAYSHOR,1759.0,NaN,1982.0,NaN,853.0,2937.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2575-S-Bayshor...,MARMLS,A11657621,N,Y,25.732866,-80.233547,https://www.redfin.com/FL/Miami/2575-S-Bayshor...,Scott Shuffield,BHHS EWM Realty,None,None,Rachelle Kaplan,"Kerdyk Real Estate, Inc.",None,None,6,blue
6,PAST SALE,2024-10-25,Condo/Co-op,16400 Collins Ave #2641,Sunny Isles Beach,FL,33160.0,1470000.0,3.0,3.0,OCEANIA IV CONDO,2340.0,NaN,1995.0,NaN,628.0,4390.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11641402,N,Y,25.928467,-80.123797,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Marcelo Kohen,One Sotheby's International Realty,None,None,Horacio Rosito,Tectum LLC,None,None,7,blue
7,PAST SALE,2024-10-22,Condo/Co-op,18911 Collins Ave #2704,Sunny Isles Beach,FL,33160.0,1445000.0,2.0,2.5,OCEAN THREE CONDO,1903.0,NaN,2003.0,NaN,759.0,1350.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11653936,N,Y,25.950878,-80.119958,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Lana Levitina,Coldwell Banker Realty,None,None,None,None,None,None,8,blue
8,PAST SALE,2024-10-23,Condo/Co-op,1581 Brickell Ave #1002,Miami,FL,33129.0,1230000.0,2.0,2.0,VILLA REGINA CONDO,1795.0,NaN,1982.0,NaN,685.0,2180.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1581-Brickell-...,MARMLS,A11592923,N,Y,25.756406,-80.193330,https://www.redfin.com/FL/Miami/1581-Brickell-...,Maricela Maury,The Corcor

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-21,Condo/Co-op,1451 Brickell Ave Ph 56,Miami,FL,33131.0,33000000.0,5.0,7.5,ECHO BRICKELL CONDO,11068.0,NaN,2017.0,NaN,2982.0,17812.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11515752,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Chad Carroll,"Compass Florida, LLC",Matthew Dugow,"Compass Florida, LLC.",Stephan Hermida,Engel & Völkers Miami,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_10282024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'ECHO BRICKELL CONDO at 1451 Brickell Ave Ph 56 in Miami'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-21,Condo/Co-op,1451 Brickell Ave Ph 56,Miami,FL,33131.0,33000000.0,5.0,7.5,ECHO BRICKELL CONDO,11068.0,NaN,2017.0,NaN,2982.0,17812.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11515752,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Chad Carroll,"Compass Florida, LLC",Matthew Dugow,"Compass Florida, LLC.",Stephan Hermida,Engel & Völkers Miami,None,None,1,orange,ECHO BRICKELL CONDO at 1451 Brickell Ave Ph 56...
1,PAST SALE,2024-10-25,Condo/Co-op,700 NE 26th Ter #2302,Miami,FL,33137.0,2300000.0,2.0,3.5,EDGEWATER CONDO,2347.0,NaN,2021.0,NaN,980.0,3696.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11672925,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Chad Carroll,"Compass Florida, LLC",None,None,Rana Tilllawi,"Compass Florida, LLC.",None,None,2,blue,EDGEWATER CONDO at 700 NE 26th Ter #2302 in Miami
2,PAST SALE,2024-10-22,Condo/Co-op,1451 Brickell Ave #3001,Miami,FL,33131.0,2300000.0,2.0,3.0,ECHO BRICKELL CONDO,1637.0,NaN,2017.0,NaN,1405.0,3785.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11537487,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Allan Filgueiras,AF Realty,None,None,None,None,None,None,3,blue,ECHO BRICKELL CONDO at 1451 Brickell Ave #3001...
3,PAST SALE,2024-10-21,Condo/Co-op,900 Brickell Key Blvd #401,Miami,FL,33131.0,2000000.0,3.0,3.5,ASIA CONDO,2065.0,NaN,2008.0,NaN,969.0,3412.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11613137,N,Y,25.768844,-80.185937,https://www.redfin.com/FL/Miami/900-Brickell-K...,Rita Herrera,London Foster Realty,None,None,Rita Herrera,London Foster Realty,None,None,4,blue,ASIA CONDO at 900 Brickell Key Blvd #401 in Miami
4,PAST SALE,2024-10-25,Condo/Co-op,480 NE 31st St #4801,Miami,FL,33137.0,1950000.0,4.0,4.5,GRAN PARAISO CONDO,1871.0,NaN,2019.0,NaN,1042.0,1920.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/480-NE-31st-St...,MARMLS,A11448155,N,Y,25.806186,-80.187176,https://www.redfin.com/FL/Miami/480-NE-31st-St...,Fabio Visconti,"Rising Global Realty, LLC.",None,None,Adam Redolfi,Alure Capital,None,None,5,blue,GRAN PARAISO CONDO at 480 NE 31st St #4801 in ...
5,PAST SALE,2024-10-21,Condo/Co-op,2575 S Bayshore Dr Unit 9B,Miami,FL,33133.0,1500000.0,3.0,2.0,THE COCONUT GROVE BAYSHOR,1759.0,NaN,1982.0,NaN,853.0,2937.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2575-S-Bayshor...,MARMLS,A11657621,N,Y,25.732866,-80.233547,https://www.redfin.com/FL/Miami/2575-S-Bayshor...,Scott Shuffield,BHHS EWM Realty,None,None,Rachelle Kaplan,"Kerdyk Real Estate, Inc.",None,None,6,blue,THE COCONUT GROVE BAYSHOR at 2575 S Bayshore D...
6,PAST SALE,2024-10-25,Condo/Co-op,16400 Collins Ave #2641,Sunny Isles Beach,FL,33160.0,1470000.0,3.0,3.0,OCEANIA IV CONDO,2340.0,NaN,1995.0,NaN,628.0,4390.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11641402,N,Y,25.928467,-80.123797,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Marcelo Kohen,One Sotheby's International Realty,None,None,Horacio Rosito,Tectum LLC,None,None,7,blue,OCEANIA IV CONDO at 16400 Collins Ave #2641 in...
7,PAST SALE,2024-10-22,Condo/Co-op,18911 Collins Ave #2704,Sunny Isles Beach,FL,33160.0,1445000.0,2.0,2.5,OCEAN THREE CONDO,1903.0,NaN,2003.0,NaN,759.0,1350.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11653936,N,Y,25.950878,-80.119958,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Lana Levitina,Coldwell Banker Realty,None,

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: ECHO BRICKELL CONDO closing tops Miami Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,040,000M to $33,000,000M
FEATURED HED:
SEO HED: Miami Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami Dade, weekly condo sales, Miami, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami Dade County’s NEWS PEG HERE.

Brokers closed 112 condo sales totaling $87,665,799 million from Oct. 20th to Oct. 27th. The previous week, brokers closed 111 condo sales totaling $78,400,000.

Last week’s units sold for an average of $782,730, higher than the $706,659 average price from the previous week. The average price per square foot fell to $478 f

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-21,Condo/Co-op,1451 Brickell Ave Ph 56,Miami,FL,33131.0,33000000.0,5.0,7.5,ECHO BRICKELL CONDO,11068.0,NaN,2017.0,NaN,2982.0,17812.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11515752,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Chad Carroll,"Compass Florida, LLC",Matthew Dugow,"Compass Florida, LLC.",Stephan Hermida,Engel & Völkers Miami,None,None,1,orange,ECHO BRICKELL CONDO at 1451 Brickell Ave Ph 56...
1,PAST SALE,2024-10-25,Condo/Co-op,700 NE 26th Ter #2302,Miami,FL,33137.0,2300000.0,2.0,3.5,EDGEWATER CONDO,2347.0,NaN,2021.0,NaN,980.0,3696.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11672925,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Chad Carroll,"Compass Florida, LLC",None,None,Rana Tilllawi,"Compass Florida, LLC.",None,None,2,blue,EDGEWATER CONDO at 700 NE 26th Ter #2302 in Miami
2,PAST SALE,2024-10-22,Condo/Co-op,1451 Brickell Ave #3001,Miami,FL,33131.0,2300000.0,2.0,3.0,ECHO BRICKELL CONDO,1637.0,NaN,2017.0,NaN,1405.0,3785.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11537487,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Allan Filgueiras,AF Realty,None,None,None,None,None,None,3,blue,ECHO BRICKELL CONDO at 1451 Brickell Ave #3001...
3,PAST SALE,2024-10-21,Condo/Co-op,900 Brickell Key Blvd #401,Miami,FL,33131.0,2000000.0,3.0,3.5,ASIA CONDO,2065.0,NaN,2008.0,NaN,969.0,3412.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11613137,N,Y,25.768844,-80.185937,https://www.redfin.com/FL/Miami/900-Brickell-K...,Rita Herrera,London Foster Realty,None,None,Rita Herrera,London Foster Realty,None,None,4,blue,ASIA CONDO at 900 Brickell Key Blvd #401 in Miami
4,PAST SALE,2024-10-25,Condo/Co-op,480 NE 31st St #4801,Miami,FL,33137.0,1950000.0,4.0,4.5,GRAN PARAISO CONDO,1871.0,NaN,2019.0,NaN,1042.0,1920.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/480-NE-31st-St...,MARMLS,A11448155,N,Y,25.806186,-80.187176,https://www.redfin.com/FL/Miami/480-NE-31st-St...,Fabio Visconti,"Rising Global Realty, LLC.",None,None,Adam Redolfi,Alure Capital,None,None,5,blue,GRAN PARAISO CONDO at 480 NE 31st St #4801 in ...
5,PAST SALE,2024-10-21,Condo/Co-op,2575 S Bayshore Dr Unit 9B,Miami,FL,33133.0,1500000.0,3.0,2.0,THE COCONUT GROVE BAYSHOR,1759.0,NaN,1982.0,NaN,853.0,2937.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2575-S-Bayshor...,MARMLS,A11657621,N,Y,25.732866,-80.233547,https://www.redfin.com/FL/Miami/2575-S-Bayshor...,Scott Shuffield,BHHS EWM Realty,None,None,Rachelle Kaplan,"Kerdyk Real Estate, Inc.",None,None,6,blue,THE COCONUT GROVE BAYSHOR at 2575 S Bayshore D...
6,PAST SALE,2024-10-25,Condo/Co-op,16400 Collins Ave #2641,Sunny Isles Beach,FL,33160.0,1470000.0,3.0,3.0,OCEANIA IV CONDO,2340.0,NaN,1995.0,NaN,628.0,4390.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11641402,N,Y,25.928467,-80.123797,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Marcelo Kohen,One Sotheby's International Realty,None,None,Horacio Rosito,Tectum LLC,None,None,7,blue,OCEANIA IV CONDO at 16400 Collins Ave #2641 in...
7,PAST SALE,2024-10-22,Condo/Co-op,18911 Collins Ave #2704,Sunny Isles Beach,FL,33160.0,1445000.0,2.0,2.5,OCEAN THREE CONDO,1903.0,NaN,2003.0,NaN,759.0,1350.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11653936,N,Y,25.950878,-80.119958,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Lana Levitina,Coldwell Banker Realty,None,

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: ECHO BRICKELL CONDO closing tops Miami Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,040,000M to $33,000,000M
FEATURED HED:
SEO HED: Miami Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami Dade, weekly condo sales, Miami, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami Dade County’s NEWS PEG HERE.

Brokers closed 112 condo sales totaling $87,665,799 million from Oct. 20th to Oct. 27th. The previous week, brokers closed 111 condo sales totaling $78,400,000.

Last week’s units sold for an average of $782,730, higher than the $706,659 average price from the previous week. The average price per square foot fell to $478 f

In [62]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami/1451-Brickell-Ave-33131/unit-56/home/56742997


In [63]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami/700-NE-26th-Ter-33137/unit-2302/home/193146621


In [64]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/900-Brickell-Key-Blvd-33131/unit-1805/home/43412291


In [65]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/1581-Brickell-Ave-33129/unit-1002/home/42764244


In [66]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami/1451-Brickell-Ave-33131/unit-56/home/56742997


In [67]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/16400-Collins-Ave-33160/unit-2641/home/43247797


In [68]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-21,Condo/Co-op,1451 Brickell Ave Ph 56,Miami,FL,33131.0,33000000.0,5.0,7.5,ECHO BRICKELL CONDO,11068.0,NaN,2017.0,NaN,2982.0,17812.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11515752,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Chad Carroll,"Compass Florida, LLC",Matthew Dugow,"Compass Florida, LLC.",Stephan Hermida,Engel & Völkers Miami,None,None,1,orange,ECHO BRICKELL CONDO at 1451 Brickell Ave Ph 56...
1,PAST SALE,2024-10-25,Condo/Co-op,700 NE 26th Ter #2302,Miami,FL,33137.0,2300000.0,2.0,3.5,EDGEWATER CONDO,2347.0,NaN,2021.0,NaN,980.0,3696.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11672925,N,Y,25.802013,-80.185776,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Chad Carroll,"Compass Florida, LLC",None,None,Rana Tilllawi,"Compass Florida, LLC.",None,None,2,blue,EDGEWATER CONDO at 700 NE 26th Ter #2302 in Miami
2,PAST SALE,2024-10-22,Condo/Co-op,1451 Brickell Ave #3001,Miami,FL,33131.0,2300000.0,2.0,3.0,ECHO BRICKELL CONDO,1637.0,NaN,2017.0,NaN,1405.0,3785.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1451-Brickell-...,MARMLS,A11537487,N,Y,25.758141,-80.192431,https://www.redfin.com/FL/Miami/1451-Brickell-...,Allan Filgueiras,AF Realty,None,None,None,None,None,None,3,blue,ECHO BRICKELL CONDO at 1451 Brickell Ave #3001...
3,PAST SALE,2024-10-21,Condo/Co-op,900 Brickell Key Blvd #401,Miami,FL,33131.0,2000000.0,3.0,3.5,ASIA CONDO,2065.0,NaN,2008.0,NaN,969.0,3412.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11613137,N,Y,25.768844,-80.185937,https://www.redfin.com/FL/Miami/900-Brickell-K...,Rita Herrera,London Foster Realty,None,None,Rita Herrera,London Foster Realty,None,None,4,blue,ASIA CONDO at 900 Brickell Key Blvd #401 in Miami
4,PAST SALE,2024-10-25,Condo/Co-op,480 NE 31st St #4801,Miami,FL,33137.0,1950000.0,4.0,4.5,GRAN PARAISO CONDO,1871.0,NaN,2019.0,NaN,1042.0,1920.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/480-NE-31st-St...,MARMLS,A11448155,N,Y,25.806186,-80.187176,https://www.redfin.com/FL/Miami/480-NE-31st-St...,Fabio Visconti,"Rising Global Realty, LLC.",None,None,Adam Redolfi,Alure Capital,None,None,5,blue,GRAN PARAISO CONDO at 480 NE 31st St #4801 in ...
5,PAST SALE,2024-10-21,Condo/Co-op,2575 S Bayshore Dr Unit 9B,Miami,FL,33133.0,1500000.0,3.0,2.0,THE COCONUT GROVE BAYSHOR,1759.0,NaN,1982.0,NaN,853.0,2937.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2575-S-Bayshor...,MARMLS,A11657621,N,Y,25.732866,-80.233547,https://www.redfin.com/FL/Miami/2575-S-Bayshor...,Scott Shuffield,BHHS EWM Realty,None,None,Rachelle Kaplan,"Kerdyk Real Estate, Inc.",None,None,6,blue,THE COCONUT GROVE BAYSHOR at 2575 S Bayshore D...
6,PAST SALE,2024-10-25,Condo/Co-op,16400 Collins Ave #2641,Sunny Isles Beach,FL,33160.0,1470000.0,3.0,3.0,OCEANIA IV CONDO,2340.0,NaN,1995.0,NaN,628.0,4390.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11641402,N,Y,25.928467,-80.123797,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Marcelo Kohen,One Sotheby's International Realty,None,None,Horacio Rosito,Tectum LLC,None,None,7,blue,OCEANIA IV CONDO at 16400 Collins Ave #2641 in...
7,PAST SALE,2024-10-22,Condo/Co-op,18911 Collins Ave #2704,Sunny Isles Beach,FL,33160.0,1445000.0,2.0,2.5,OCEAN THREE CONDO,1903.0,NaN,2003.0,NaN,759.0,1350.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11653936,N,Y,25.950878,-80.119958,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Lana Levitina,Coldwell Banker Realty,None,

## Time on Market Calculator

In [69]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 10, 16) ## List (Earlier) date
date2 = datetime(2024, 10, 25) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

9


## Clean CSV for Datawrapper Chart

In [70]:
chart_df = df_top_ten

In [71]:
chart_df = chart_df.fillna(" ")

In [72]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [73]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [74]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [75]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [76]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [77]:
csv_date_string = today.strftime("%m_%d_%Y")

In [78]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [79]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-10-21,1451 Brickell Ave Ph 56 in Miami,"$33,000,000",5,7.5,"11,068",2017,"$2,982",MARMLS,A11515752,"Chad Carroll Compass Florida, LLC Matthew Dugo...",Stephan Hermida Engel & Völkers Miami
1,2024-10-25,700 NE 26th Ter #2302 in Miami,"$2,300,000",2,3.5,"2,347",2021,$980,MARMLS,A11672925,"Chad Carroll Compass Florida, LLC","Rana Tilllawi Compass Florida, LLC."
2,2024-10-22,1451 Brickell Ave #3001 in Miami,"$2,300,000",2,3.0,"1,637",2017,"$1,405",MARMLS,A11537487,Allan Filgueiras AF Realty,
3,2024-10-21,900 Brickell Key Blvd #401 in Miami,"$2,000,000",3,3.5,"2,065",2008,$969,MARMLS,A11613137,Rita Herrera London Foster Realty,Rita Herrera London Foster Realty
4,2024-10-25,480 NE 31st St #4801 in Miami,"$1,950,000",4,4.5,"1,871",2019,"$1,042",MARMLS,A11448155,"Fabio Visconti Rising Global Realty, LLC.",Adam Redolfi Alure Capital
5,2024-10-21,2575 S Bayshore Dr Unit 9B in Miami,"$1,500,000",3,2.0,"1,759",1982,$853,MARMLS,A11657621,Scott Shuffield BHHS EWM Realty,"Rachelle Kaplan Kerdyk Real Estate, Inc."
6,2024-10-25,16400 Collins Ave #2641 in Sunny Isles Beach,"$1,470,000",3,3.0,"2,340",1995,$628,MARMLS,A11641402,Marcelo Kohen One Sotheby's International Real...,Horacio Rosito Tectum LLC
7,2024-10-22,18911 Collins Ave #2704 in Sunny Isles Beach,"$1,445,000",2,2.5,"1,903",2003,$759,MARMLS,A11653936,Lana Levitina Coldwell Banker Realty,
8,2024-10-23,1581 Brickell Ave #1002 in Miami,"$1,230,000",2,2.0,"1,795",1982,$685,MARMLS,A11592923,Maricela Maury The Corcoran Group,"Mayra Cortes Malaret AG Real Estate Advisors, ..."
9,2024-10-21,900 Brickell Key Blvd #1805 in Miami,"$1,040,000",2,2.5,"1,326",2008,$784,MARMLS,A11595145,"Alexandra Sierra Compass Florida, LLC Maria Ma...","Alexandra Sierra Compass Florida, LLC"
